# TV Script Generation

In this project, [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts are generated using RNNs.  The data is from the kaggle [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network build here will generate a new, "fake" TV script, based on patterns it recognizes in this training data.

## Loading the Data

As a first step, the data from `./data/Seinfeld_Scripts.txt` is loaded and display some samples. 

In [ ]:
# load in data
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data
  
data_dir = './data/Seinfeld_Scripts.txt'
text = load_data(data_dir)

## Exploring the Data
Use the tuple `view_line_range` to view different parts of the data. This will give a sense of the data which will be used for training the model. It can be noticed, for example, that the data is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

Some of the statistics about the dataset could also be viewed.

In [ ]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implementing Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implementing the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, first the words are transformed to ids.  In this function, two dictionaries are created:
- Dictionary to go from the words to an id, and is called `vocab_to_int`
- Dictionary to go from the id to word, and is called `int_to_vocab`

These dictionaries are returned as the following **tuple** `(vocab_to_int, int_to_vocab)`

In [ ]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary

    Input: 
    text - The text of tv scripts split into words

    Output:
    return - A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    word_count = Counter(text)
    sorted_vocab = sorted(word_count, key=word_count.get, reverse=True)
    int_to_vocab = {pos: word for pos, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: pos for pos, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
While splitting the script into a word array using spaces as delimiters, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

So, implementing the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Creating a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. To make sure not to use a value that could be confused as a word; for example, instead of using the value "dash", using something like "||dash||".

In [ ]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    
    Output:
    return - Tokenized dictionary where the key is the punctuation and the value is the token
    """
  
    punctuation_dict = {'.':'||Period||', 
                       ',': '||Comma||',
                       '"': '||QuotationMark||',
                       ';': '||Semicolon||',
                       '!': '||ExclamationMark||',
                       '?': '||QuestionMark||',
                       '(': '||LeftParentheses||',
                       ')': '||RightParentheses||',
                       '-': '||Dash||',
                       '\n': '||Return||'}
    
    return punctuation_dict

## Pre-processing all the data and saving it

The function `preprocess_and_save_data` will pre-process all the data and save it to file 'preprocess.p'.

In [ ]:
# pre-process training data

SPECIAL_WORDS = {'PADDING': '<PAD>'}

def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    text = load_data(dataset_path)
    
    # Ignore notice, since it is not useful for analysing the data
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

In [ ]:
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is the checkpoint to load the pre-processed data directly from the saved file from the disk instead of carrying out all the above steps. The function 'load_preprocess' in the cell below loads the pre-processed data from 'preprocess.p' file.

In [ ]:
def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

In [ ]:
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

## Building the Neural Network
In this section, the components necessary to build an RNN will be built by implementing the RNN Module and forward and backpropagation functions.

#### Check Access to GPU

In [ ]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## **Input**
Let's start with the preprocessed input data. Using [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) provides a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

Creating data with TensorDataset by passing in feature and target tensors. Then creating a DataLoader.

#### Batching
Implementing the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.


For example, if the input are:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Then the first `feature_tensor` would contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` would just be the next "word"/tokenized word value:
```
5
```
This continues with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```
and so on

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    
    Input:
    words - The word ids of the TV scripts
    sequence_length - The sequence length of each batch
    batch_size - The size of each batch; the number of sequences in a batch
    
    Output:
    return - DataLoader with batched data
    """

    n_batches = len(words)//batch_size
    # only full batches
    words = words[:n_batches*batch_size]
    
    feature_tensors = []
    target_tensors = []
    i=0
    for _ in words:
        feature_tensors.append(words[:sequence_length])
        target_tensors.append(words[sequence_length])
        words = words[i+1:]
        if len(words) < sequence_length+1:
            break

    data = TensorDataset(torch.from_numpy(np.asarray(feature_tensors)), torch.from_numpy(np.asarray(target_tensors)))
    data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    
    # return a dataloader
    return data_loader

### Testing the dataloader 

Below, is some test text data and defining a dataloader using the function defined above. Then, getting some sample batch of inputs `sample_x` and targets `sample_y` from the dataloader.

The code should return something like the following to know if the implemented functions works as expected (likely in a different order, if data is shuffled):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
The sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

Notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [ ]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 20,  21,  22,  23,  24],
        [  6,   7,   8,   9,  10],
        [ 33,  34,  35,  36,  37],
        [ 40,  41,  42,  43,  44],
        [ 23,  24,  25,  26,  27],
        [ 22,  23,  24,  25,  26],
        [ 43,  44,  45,  46,  47],
        [ 32,  33,  34,  35,  36],
        [ 30,  31,  32,  33,  34],
        [  4,   5,   6,   7,   8]])

torch.Size([10])
tensor([ 25,  11,  38,  45,  28,  27,  48,  37,  35,   9])


---
## Build the Neural Network
Implementing a RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). LSTM is used here. 


The output of this model would be the *last* batch of word scores after a complete sequence has been processed. That is, for each input sequence of words, the output would be the word score for the single most likely next word.

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module

        Input:
        vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        output_size: The number of output dimensions of the neural network
        embedding_dim: The size of embeddings, should you choose to use them        
        hidden_dim: The size of the hidden layer outputs
        dropout: dropout to add in between LSTM/GRU layers
        """
        
        super(RNN, self).__init__()
        # setting class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # defining model layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.output = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network

        Input:
        nn_input: The input to the neural network
        hidden: The hidden state        
        
        Output:
        return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)
        
        nn_input = nn_input.long()
        embed_out = self.embed(nn_input)
        lstm_out, hidden = self.lstm(embed_out, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.output(lstm_out)
        out = out.view(batch_size, -1, self.output_size)
        output = out[:,-1]
        

        # returning one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM

        Input:
        batch_size: The batch_size of the hidden state
        
        Output:
        return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initializing hidden state with zero weights, and move to GPU if available
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

### Defining forward and backpropagation

The implemented RNN class is used to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it would return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`.

If a GPU is available, you should move your data to that GPU device, here.

In [ ]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    
    Input:
    decoder: The PyTorch Module that holds the neural network
    decoder_optimizer: The PyTorch optimizer for the neural network
    criterion: The PyTorch loss function
    inp: A batch of input to the neural network
    target: The target output for the batch of input
    
    Output:
    return: The loss and the latest hidden state Tensor
    """
 
    # move data to GPU, if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    inp, target = inp.to(device), target.to(device)
    
    hidden = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    # perform backpropagation and optimization
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    loss.backward()
    
    # 'clip_grad_norm' helps prevent the exploding gradient problem in RNNs / LSTMs
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    
    optimizer.step()

    # return the loss over a batch and the hidden state produced by the model
    return loss.item(), hidden

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter.

In [ ]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # to make sure to iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Training the neural network with the following parameters:
- `sequence_length` to the length of a sequence.
-  `batch_size` to the batch size.
-  `num_epochs` to the number of epochs to train for.
- `learning_rate` to the learning rate for an Adam optimizer.
-  `vocab_size` to the number of uniqe tokens in our vocabulary.
- `output_size` to the desired size of the output.
-  `embedding_dim` to the embedding dimension; smaller than the vocab_size.
-  `hidden_dim` to the hidden dimension of your RNN.
-  `n_layers` to the number of layers/cells in your RNN.
-  `show_every_n_batches` to the number of batches at which the neural network should print progress.

In [ ]:
# Data params
# Sequence Length
sequence_length = 50  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)
torch.save(train_loader, 'dataloader.pth')
torch.load('dataloader.pth')

In [ ]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
The next cell will train the neural network on the pre-processed data.

In [ ]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)

save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.463102088928222

Epoch:    1/10    Loss: 4.850352227687836

Epoch:    1/10    Loss: 4.596398238182068

Epoch:    1/10    Loss: 4.500200746059417

Epoch:    1/10    Loss: 4.386232285499573

Epoch:    1/10    Loss: 4.342856774330139

Epoch:    1/10    Loss: 4.2818917708396915

Epoch:    1/10    Loss: 4.2593809499740605

Epoch:    1/10    Loss: 4.211174781322479

Epoch:    1/10    Loss: 4.200193108558655

Epoch:    1/10    Loss: 4.141652512073517

Epoch:    1/10    Loss: 4.1599180793762205

Epoch:    1/10    Loss: 4.153071348667145

Epoch:    2/10    Loss: 3.9993373478247114

Epoch:    2/10    Loss: 3.944969476222992

Epoch:    2/10    Loss: 3.9294598569869996

Epoch:    2/10    Loss: 3.93143209695816

Epoch:    2/10    Loss: 3.9330968589782715

Epoch:    2/10    Loss: 3.9054547185897825

Epoch:    2/10    Loss: 3.9111528248786924

Epoch:    2/10    Loss: 3.9114336414337156

Epoch:    2/10    Loss: 3.8835342803001405

Epoch:    2/10   

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


---
# Checkpoint

After running the above training cell, the model will be saved by name, `trained_rnn`. This model could be used later to generate the TV scripts. 

There is a possibility to pause at this stage and resume the progress by running the next cell, which will load in the word:id dictionaries _and_ load in the saved model by name!

In [ ]:
import torch

def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

In [ ]:
_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
trained_rnn = load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, it could be used to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. the `generate` function could be used to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [ ]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network

    Input:
    decoder: The PyTorch Module that holds the trained neural network
    prime_id: The word id to start the first prediction
    int_to_vocab: Dict of word id keys to word values
    token_dict: Dict of puncuation tokens keys to puncuation values
    pad_value: The value used to pad a sequence
    predict_len: The length of text to generate
    
    Output:
    return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script to be generated and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

It is possible to set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (or start with any other names from the original text file!)

In [ ]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


elaine:(to vanessa, to vanessa)

[setting: dealership office

jerry: so?

elaine: oh, i got a date.

george: i know.

jerry: you mean you don't want a drink?

jerry: no. i can't believe it.

jerry: well, you know, i'm not going to get some of these things.

elaine: oh, you know i was in a mood of mine. i was in the mood with her. i mean, you know, i was just a little bit worried.

jerry: oh, i can't believe it. i think i'm going to be here. you know, it's like, uh- i think i'm getting it off.

jerry: what are you doing here?

elaine: well i guess i could get a call.

elaine: what?

george: yeah.

jerry: i don't know.

george: you can't take a look at the car?

jerry:(to jerry) hey hey, hey, what is the deal with this guy?

george:(to jerry) oh, no, i'm sorry, i don't want it.

george:(to kramer) oh, hi, i'm just sorry about the truth.(hangs up) you know, i think you're not really going to be in a while.

george:(to elaine) i thought you were a communist.

jerry:(pause) oh yeah?(to geor

#### Save the scripts

Once the script is generated, save it to a text file!

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()